# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

![](avocado_wallpaper.jpeg)

In [18]:
import pandas as pd

avocado = pd.read_csv('data/avocado.csv', delimiter = '\t')

# Subsetting with the relevant columns
relevant_columns = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']

avocado = avocado[relevant_columns]

avocado


,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,0819573012712,en,"Organic Baby Food, Apples, Kale & Avocados",NaN,113 g (113 g),NaN,Happybaby,happybaby,NaN,NaN,United States,en:united-states,NaN,NaN
1781,0052200072097,en,"Just Pineapple, Pear & Avocado",NaN,60 g (60 g),NaN,"Beech-Nut, Beech-Nut Nutrition Company","beech-nut,beech-nut-nutrition-company",NaN,NaN,United States,en:united-states,NaN,NaN
1782,0793613300000,en,Spinach Avocado Dip,NaN,28 g (2 Tbsp),NaN,Classy Delites,classy-delites,NaN,NaN,United States,en:united-states,NaN,NaN
1783,05252428,en,"Organic Just Apple, Raspberry & Avocado, Apple...",NaN,60 g (0.25 cup),NaN,Beech-Nut,beech-nut,NaN,NaN,United States,en:united-states,NaN,NaN


In [19]:
# Getting the informations from the txt file
with open('data/relevant_avocado_categories.txt', "r") as file:
    relevant_avocado_categories = file.read().splitlines()
    file.close()
    
relevant_avocado_categories

['en:avocadoes',
 'en:avocados',
 'en:fresh-foods',
 'en:fresh-vegetables',
 'en:fruchte',
 'en:fruits',
 'en:raw-green-avocados',
 'en:tropical-fruits',
 'en:tropische-fruchte',
 'en:vegetables-based-foods',
 'fr:hass-avocados']

In [20]:
avocado.head()

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


In [21]:
avocado['categories_list'] = avocado['categories_tags'].str.split(',')
avocado = avocado.dropna(subset = 'categories_list')
# Filtering based off of the txt file categories
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_avocado_categories]))]

avocado

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags,categories_list
5,3662994002063,fr,NaN,3 fruits,NaN,NaN,la compagnie des fruits mûrs,la-compagnie-des-fruits-murs,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
6,8437013031011,fr,NaN,1 kg,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
14,4016249238155,de,NaN,135g,100g,de:gläschen,Allos,allos,"en:plant-based-foods-and-beverages,en:plant-ba...","en:organic,en:vegetarian,en:eu-organic,en:no-g...",Deutschland,en:germany,Europäische Union,en:european-union,"[en:plant-based-foods-and-beverages, en:plant-..."
17,8718963381532,de,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Deutschland,en:germany,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
23,8436002746707,es,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,España,en:spain,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,3700353611218,fr,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
1756,4311527575718,de,NaN,NaN,NaN,de:aufkleber,Edeka,edeka,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Deutschland,en:germany,Peru,en:peru,"[en:plant-based-foods-and-beverages, en:plant-..."
1757,4311527571499,en,NaN,NaN,NaN,en:aufkleber,Edeka,edeka,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Germany,en:germany,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."
1769,3439496511399,fr,NaN,NaN,NaN,NaN,NaN,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,France,en:france,NaN,NaN,"[en:plant-based-foods-and-beverages, en:plant-..."


In [22]:
# Filter for when the country receiving is United Kingdom
top_avocado = avocado[avocado['countries'] == 'United Kingdom']
# Counting the country of origin from which we get the most avocados
top_avocado_origin = top_avocado['origins_tags'].value_counts().index[0]

top_avocado_origin = top_avocado_origin.lstrip('en:')

top_avocado_origin


'peru'

In [23]:
def read_and_filter_data(filename, txt_file):
    # Read the file
    df = pd.read_csv(filename, delimiter = '\t')
    
    # Getting the informations from the txt file
    with open(txt_file, "r") as file:
        relevant_categories = file.read().splitlines()
        file.close()
    
    # Filtering for the relevant categories
    df['categories_list'] = df['categories_tags'].str.split(',')
    df = df.dropna(subset = 'categories_list')
    # Filtering based off of the txt file categories
    df = df[df['categories_list'].apply(lambda x: any([i for i in x if i in relevant_categories]))]
    
    # Filter for when the country receiving is United Kingdom
    top = df[df['countries'] == 'United Kingdom']
    # Counting the country of origin from which we get the most avocados
    top_origin = top['origins_tags'].value_counts().index[0]

    top_origin = top_origin.lstrip('en:')
    top_origin = top_origin.replace('-', ' ')
   

    return top_origin

# Making sure the function works
read_and_filter_data('data/avocado.csv', 'data/relevant_avocado_categories.txt')

'peru'

In [24]:
# Executing for oil and sourdough
top_olive_oil_origin = read_and_filter_data('data/olive_oil.csv', 'data/relevant_olive_oil_categories.txt')

print(top_olive_oil_origin)

top_sourdough_origin = read_and_filter_data('data/sourdough.csv', 'data/relevant_sourdough_categories.txt')

print(top_sourdough_origin)

greece
united kingdom
